In [1]:
import numpy as np
import plotly
import plotly.graph_objects
import plotly.express as px
import ipywidgets
import IPython.display


def parabola(x, y, slider_value=1.0):
    return slider_value*x*x+y*x


def square(x, y, slider_value=1.0):
    return slider_value*((x-y))**2


min_x = -5.0
max_x = 5.0
n_slices = 10
n_scales = 5

figure = plotly.graph_objects.FigureWidget(
    [plotly.graph_objects.Surface(), plotly.graph_objects.Surface(), plotly.graph_objects.Surface(), plotly.graph_objects.Surface(), plotly.graph_objects.Surface(), plotly.graph_objects.Scatter3d()] + [
        plotly.graph_objects.Scatter3d() for _ in range(2*n_slices)
    ] + [
        plotly.graph_objects.Surface() for _ in range(n_scales)
    ], plotly.graph_objects.Layout(scene={'xaxis_title': "X axis", 'yaxis_title': "Y axis", 'zaxis_title': "Z axis"}),
)
figure.update_layout(showlegend=False)


def update_surface(change):
    selected_value = change["new"]
    ox = np.linspace(min_x, max_x, 50)
    oy = np.linspace(min_x, max_x, 50)
    xy, yx = np.meshgrid(ox, oy)
    z_parabola = parabola(xy, yx)
    square_parabola = square(xy, yx)
    selected_id = np.searchsorted(oy, selected_value)
    max_z = z_parabola.max()
    with figure.batch_update():
        surface = figure.data[3]
        surface.colorscale = px.colors.sequential.Inferno
        surface.opacity = 0.7
        surface.x = xy
        surface.y = yx
        surface.z = z_parabola

        slider_parabola = square(xy, yx)
        selected_scatter = figure.data[4]
        selected_scatter.colorscale = px.colors.sequential.Viridis
        selected_scatter.opacity = 0.7
        selected_scatter.x = xy
        selected_scatter.y = yx
        selected_scatter.z = square_parabola

        XoY = figure.data[0]
        XoY.colorscale = px.colors.sequential.Inferno
        XoY.opacity = 0.2
        XoY.surfacecolor = xy + yx
        XoY.x = xy
        XoY.y = yx
        XoY.z = np.zeros_like(xy)

        min_z = min(z_parabola.min(), slider_parabola.min())
        max_z = max(z_parabola.max(), slider_parabola.max())
        oz = np.linspace(min_z, max_z, 2)
        plane_xz, plane_zx = np.meshgrid(ox, oz)
        XoZ = figure.data[1]
        XoZ.colorscale = px.colors.sequential.Viridis
        XoZ.opacity = 0.2
        XoZ.surfacecolor = plane_xz + plane_zx
        XoZ.x = plane_xz
        XoZ.y = np.zeros_like(plane_xz)
        XoZ.z = plane_zx

        YoZ = figure.data[2]
        YoZ.colorscale = px.colors.sequential.Cividis
        YoZ.opacity = 0.2
        YoZ.surfacecolor = plane_xz + plane_zx
        YoZ.x = np.zeros_like(plane_zx)
        YoZ.y = plane_xz
        YoZ.z = plane_zx

        scatter = figure.data[5]
        scatter.mode="lines"
        scatter.line=dict(color='red', width=4)
        argmin = slider_parabola[selected_id].argmin()
        scatter.x = np.full([2], ox[argmin])
        scatter.y=[selected_value, selected_value]
        scatter.z=[slider_parabola[selected_id, argmin], max_z]

        traces_iter = iter(figure.data)
        next(traces_iter)
        next(traces_iter)
        next(traces_iter)
        next(traces_iter)
        next(traces_iter)
        next(traces_iter)
        for y_coordinate in np.linspace(min_x, max_x, n_slices):
                expanded_y_temp = np.ones_like(xy) * y_coordinate
                z_2d_temp = parabola(xy, expanded_y_temp)
                scatter = next(traces_iter)
                scatter.x = xy.flatten().copy()
                scatter.y=expanded_y_temp.flatten().copy()
                scatter.z=z_2d_temp.flatten().copy()
                scatter.mode="lines"
                scatter.line=dict(color="black", width=1)
                scatter = next(traces_iter)
                argmin = z_2d_temp.argmin()
                scatter.x=[xy.flatten()[argmin], xy.flatten()[argmin]]
                scatter.y=[y_coordinate, y_coordinate]
                scatter.z=[z_2d_temp.flatten()[argmin], max_z]
                scatter.mode="lines"
                scatter.line=dict(color='black', width=1)
        surface_id = 0
        for surface in traces_iter:
            surface_color = plotly.colors.sample_colorscale(px.colors.sequential.Inferno, surface_id / (n_scales-1))[0]
            surface.colorscale = [(0.0, surface_color), (1.0, surface_color)]
            surface.opacity = 0.7
            surface.x = xy
            surface.y = yx
            surface.z = parabola(xy, yx, surface_id + 1)
            surface_id += 1
    with figure.batch_update():
        min_z = 9e9
        max_z = -9e9
        for surface in figure.data:
            if not isinstance(surface.z, tuple) and surface.z is not None:
                min_z = min(min_z, surface.z.min())
                max_z = max(max_z, surface.z.max())
        oz = np.linspace(min_z, max_z, 2)
        plane_xz, plane_zx = np.meshgrid(ox, oz)
        XoZ = figure.data[1]
        XoZ.colorscale = px.colors.sequential.Viridis
        XoZ.opacity = 0.2
        XoZ.surfacecolor = plane_xz + plane_zx
        XoZ.x = plane_xz
        XoZ.y = np.zeros_like(plane_xz)
        XoZ.z = plane_zx

        YoZ = figure.data[2]
        YoZ.colorscale = px.colors.sequential.Cividis
        YoZ.opacity = 0.2
        YoZ.surfacecolor = plane_xz + plane_zx
        YoZ.x = np.zeros_like(plane_zx)
        YoZ.y = plane_xz
        YoZ.z = plane_zx


update_surface({'new': 0.0})
slider = ipywidgets.FloatSlider(min=-10.0, max=10.0, step=1.0)
slider.observe(update_surface, names="value")
IPython.display.display(slider, figure)

FloatSlider(value=0.0, max=10.0, min=-10.0, step=1.0)

FigureWidget({
    'data': [{'colorscale': [[0.0, '#000004'], [0.1111111111111111, '#1b0c41'],
                             [0.2222222222222222, '#4a0c6b'], [0.3333333333333333,
                             '#781c6d'], [0.4444444444444444, '#a52c60'],
                             [0.5555555555555556, '#cf4446'], [0.6666666666666666,
                             '#ed6925'], [0.7777777777777778, '#fb9b06'],
                             [0.8888888888888888, '#f7d13d'], [1.0, '#fcffa4']],
              'opacity': 0.2,
              'surfacecolor': {'bdata': ('AAAAAAAAJMDmFLycgpcjwMwpeDkFLy' ... 'g5BS8jQOYUvJyClyNAAAAAAAAAJEA='),
                               'dtype': 'f8',
                               'shape': '50, 50'},
              'type': 'surface',
              'uid': '4cbc1c6b-9315-4dd2-bbe0-6d395cd6d439',
              'x': {'bdata': ('AAAAAAAAFMDMKXg5BS8TwJhT8HIKXh' ... 'ByCl4SQMwpeDkFLxNAAAAAAAAAFEA='),
                    'dtype': 'f8',
                    'shape': '50, 50'},

TODO: y=x*x for complex numbers with 4D rotation: https://youtu.be/ZmRK9J3GkhM?si=1A2ZH1RjkwmdN6z